Notebook to locally examine the fiducial KiDS1000 .tfrecords

Based off https://cosmo-gitlab.phys.ethz.ch/jafluri/cosmogrid_kids1000/-/blob/master/kids1000_analysis/input_pipeline.py

In [1]:
import tensorflow as tf
from kids1000_analysis import data, constants, input_pipeline

Instructions for updating:
`MultivariateNormalFullCovariance` is deprecated, use `MultivariateNormalTriL(loc=loc, scale_tril=tf.linalg.cholesky(covariance_matrix))` instead.
Metal device set to: Apple M1 Max

systemMemory: 32.00 GB
maxCacheSize: 10.67 GB



2023-01-05 11:21:06.298983: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-01-05 11:21:06.299132: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [2]:
tfr_file = "/Users/arne/data/KiDS1000/fiducial/fiducial_data_000.tfrecord"

In [3]:
dset_signal = tf.data.TFRecordDataset(tfr_file)
dset_signal = data.decode_dset(dset_signal, tag="patch", num_tags=15, shapes=(constants.n_pix_padded, 2*constants.n_z_bins))

# add noise, remove mean, ...

for temp in dset_signal.take(1):
    print(type(temp))
    print(temp.keys())
    print(temp["patch_0"].shape)

2023-01-05 11:21:06.552064: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


<class 'dict'>
dict_keys(['patch_0', 'patch_1', 'patch_10', 'patch_11', 'patch_12', 'patch_13', 'patch_14', 'patch_2', 'patch_3', 'patch_4', 'patch_5', 'patch_6', 'patch_7', 'patch_8', 'patch_9'])
(149504, 10)


# non-batched

In [4]:
dset_concat = data.concat_tag(dset_signal, tag="patch", num_tags=15)

for temp in dset_concat.take(1):
    print(type(temp))
    print(temp.shape)

<class 'tensorflow.python.framework.ops.EagerTensor'>
(2242560, 10)


# batched

In [5]:
dset_batched = dset_signal.batch(3)

for temp in dset_batched.take(1):
    print(type(temp))
    print(temp.keys())
    print(temp["patch_0"].shape)

<class 'dict'>
dict_keys(['patch_0', 'patch_1', 'patch_10', 'patch_11', 'patch_12', 'patch_13', 'patch_14', 'patch_2', 'patch_3', 'patch_4', 'patch_5', 'patch_6', 'patch_7', 'patch_8', 'patch_9'])
(3, 149504, 10)


In [6]:
dset_concat = data.concat_tag(dset_batched, tag="patch", num_tags=15)

for temp in dset_concat.take(1):
    print(type(temp))
    print(temp.shape)

<class 'tensorflow.python.framework.ops.EagerTensor'>
(45, 149504, 10)


# original

In [7]:
local_batch_size = 3
tags = [0]
for i in range(7):
    tags.append(2 * i + 1)
    tags.append(2 * i + 2)


dset = input_pipeline.get_train_dset(
    batch_size=3,
    signal_file_patern="/Users/arne/data/KiDS1000/fiducial/fiducial_data_000.tfrecord",
    noise_file_patern="/Users/arne/data/KiDS1000/fiducial/noise_data_000.tfrecord",
    data_file="/Users/arne/data/KiDS1000/KiDS1000_pixel_data.hdf5",
    with_bary=True,
    m_bias=1,
    c_bias=0,
    node_id=0,
    n_node=1,
    num_readers=1,
    noise_scale=1,
    tags=tags,
    set_for_eval=True,
)


In [8]:
for temp in dset.take(1):
    print(type(temp))
    print(temp.shape)

<class 'tensorflow.python.framework.ops.EagerTensor'>
(45, 149504, 10)
